# BASIC SCRIPTING OF SPOT CAPABILITIES

## Establish connection

In [1]:
!pip install bosdyn-client bosdyn-core bosdyn-api bosdyn-mission

  Using cached bosdyn_client-3.3.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached bosdyn_core-3.3.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached bosdyn_api-3.3.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached bosdyn_mission-3.3.2-py3-none-any.whl.metadata (1.8 kB)
  Using cached PyJWT-2.8.0-py3-none-any.whl.metadata (4.2 kB)
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl.metadata (5.4 kB)
  Using cached protobuf-4.24.4-cp37-abi3-macosx_10_9_universal2.whl.metadata (540 bytes)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 1.3 MB/s eta 0:00:0000:01
Using cached bosdyn_client-3.3.2-py3-none-any.whl (262 kB)
Using cached bosdyn_core-3.3.2-py3-none-any.whl (31 kB)
Using cached bosdyn_api-3.3.2-py3-none-any.whl (334 kB)
Using cached bosdyn_mission-3.3.2-py3-none-any.whl (15 kB)
Using cached Deprecated-1.2.14-py2.py3-none-any.whl (9.6 kB)
Using cached protobuf-4.24.4-cp37-abi3-macosx_10_9_universal2.whl (409 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/6

In [2]:
from bosdyn.client import Robot
from bosdyn.client.spot import Spot

# Replace these with your robot's information
robot_ip = "192.168.80.3"
username = "spot"
password = "Merkleb0t"

# Create a Robot object
robot = Robot(robot_ip)
robot.authenticate(username, password)

# Create a Spot object for interacting with Spot-specific features
spot = robot.ensure_client(Spot.client_name)


ModuleNotFoundError: No module named 'bosdyn.client.spot'

spot-BD-02300008     02-20224-6c62   strelka    spot (V3)
 Software: 3.3.2 (5ce893f904ec34e2e31f2aea32602bd3bb0e8124 2023-09-11 10:54:05)

## Basic info about robot status

In [ ]:
# Get the robot state
robot_state = robot.get_robot_state()

# Access robot information
print(f"Robot Pose: {robot_state.kinematic_state.pose}")
print(f"Battery Voltage: {robot_state.power_state.battery_voltage}")


## Control the robot

In [ ]:
from bosdyn.client.spot import power_command_pb2

# Command Spot to stand
stand_command = power_command_pb2.StandCommand.Request()
spot.power_command(stand_command)


## Robot locomation

In [ ]:
from bosdyn.client.lease import LeaseClient
from bosdyn.api import trajectory_pb2

# Acquire a lease to control the robot
lease_client = robot.ensure_client(LeaseClient.default_service_name)
lease = lease_client.acquire()

# Create a trajectory for walking forward
forward_trajectory = trajectory_pb2.Trajectory(  # Define your desired trajectory
    sequence=[...]
)

# Execute the trajectory
spot.robot_command(client=robot, command=forward_trajectory)

# Release the lease when done
lease_client.return_lease(lease)


# MOVE_TEST.py

In [ ]:
import bosdyn.client
from bosdyn.client.robot_command import RobotCommandClient, blocking_stand
from bosdyn.client.robot_command import RobotCommandBuilder
from bosdyn.geometry import EulerZXY
import time

# ENTER YOUR AUTH DATA HERE
ROBOT_IP="192.168.50.3"
# ROBOT_IP="localhost:2000"
SPOT_PASSWORD = "2zqa8dgw7lor"
SPOT_USERNAME = "admin"


# Helpers to control camera drawing (you don't need to modify it)

# Start with registering out SDK
sdk = bosdyn.client.create_standard_sdk('LessonOneClient')

# Create instance of robot and auth with credentials
robot = sdk.create_robot(ROBOT_IP)
robot.authenticate(SPOT_USERNAME, SPOT_PASSWORD)

# Create lease client and take exclusive control over Spot.
lease_client = robot.ensure_client('lease')
lease = lease_client.take()
lease_keep_alive = bosdyn.client.lease.LeaseKeepAlive(lease_client)

# Try to power on the robot
robot.power_on(timeout_sec=20)
if robot.is_powered_on():
    print("Powered On")
        # If everything went smooth, Spot face lights should turn green
else:
        # In case of some problems, e.g. somebody stole control over robot
    print("Failed")
    exit(0)

# Synchronize Spor inner time with ours - to avoid outdated commands
robot.time_sync.wait_for_sync()

# To execute robot movement, create command client through which orders are sent
command_client = robot.ensure_client(RobotCommandClient.default_service_name)

# Start movement with simple stand up
blocking_stand(command_client, timeout_sec=10)

# footprint_R_body = EulerZXY(yaw=0.1, roll=0.1, pitch=0.1)
# cmd = RobotCommandBuilder.synchro_stand_command(footprint_R_body=footprint_R_body)
# command_client.robot_command(cmd)

cmd = RobotCommandBuilder.synchro_velocity_command(-0.2, -0.2, 0)
command_client.robot_command(cmd, end_time_secs=time.time() + 5)
time.sleep(8)
cmd = RobotCommandBuilder.synchro_velocity_command(0, 0, 0)
# command_client.robot_command(cmd, timeout=10)
time.sleep(5)
# Turn off the robot gracefully
# robot.power_off(cut_immediately=False)

In [4]:
import subprocess

# SSH connection information
robot_ip="192.168.50.3"
# ROBOT_IP="localhost:2000"
robot_username = "admin"
robot_password = "2zqa8dgw7lor"


# SSH command to start streaming and forwarding the camera feed
ssh_command = (
    f"sshpass -p {robot_password} ssh {robot_username}@{robot_ip} "
    "'ffmpeg -f v4l2 -video_size 1280x720 -i /dev/video0 -f mpegts -codec:v mpeg1video -s 1280x720 -b:v 800k -bf 0 -r 30 - | tee -a /dev/null'"
)

# Open a local video player to display the stream
video_player = subprocess.Popen(["ffplay", "-"])

# Start the SSH command to stream video
ssh_process = subprocess.Popen(ssh_command, shell=True)

# Wait for the user to press Enter to stop streaming
input("Press Enter to stop streaming...")

# Terminate the SSH and video player processes
ssh_process.terminate()
video_player.terminate()


FileNotFoundError: [Errno 2] No such file or directory: 'ffplay'